Partie 2: Etude et résolution numérique

Etude du problème:

On a : $coût(t0,tf)=\int_0^T p(t)P_{fournie}(t,t0,tf)dt$

avec T la durée d'étude sur le rechargement
Cette fonction est continue sur$[0;T]^2\cap (t0<=tf)$
Donc d'après le théorème de Weierstrass, min coût(t0,tf) existe.
Ce minimum n'est clairement pas unique. Imaginons que le client veut faire rechrager sa voiture pendant les heures creuses, si la recharge n'est pas trop importante, on peut choisir l'heure pendant laquelle faire la recharge et le prix sera inchangé.



Pour mon algorithme, j'ai choisi de faire deux fonctions. Une première qui choisit les temps de début et de fin de rechargement les plus adaptés. La seconde calcule alors le coût de la recharge.

In [1]:
def choixtemps(t0,tf,pmax,SOCi,SOCf,T,Qmax):
    t1=0
    t2=0
    if tf<=T/2:
        t1=t0
        t2=tf
    else:
        t1=T/2
        t2=tf
    if (SOCf-SOCi)*Qmax/(t2-t1)>pmax:
        t1=t2-(SOCf-SOCi)*Qmax/pmax
    return(t1,t2)
    
def cout(t0,tf,SOCi,SOCf,Qmax,c1,c2,T,pmax):
    t1,t2=choixtemps(t0,tf,pmax,SOCi,SOCf,T,Qmax)
    if t2<=T/2:
        return c1*Qmax*(SOCf-SOCi)
    else:
        if t1>=T/2:
            return c2*Qmax*(SOCf-SOCi)
        else:
            return pmax(c1(T/2-t1)+c2(t2-T/2))
    
        

In [6]:

choixtemps(40000,70000,6000,0.2,0.8,86400,510)


(43200.0, 70000)

In [7]:
cout(40000,70000,0.2,0.8,510,0.1580,0.1230,86400,6000)

37.638000000000005

In [8]:
def coutnonopti(t0,tf,c1,c2,T,Qmax,SOCi,SOCf):
    p=(SOCf-SOCi)*Qmax/(tf-t0)
    return (T/2-t0)*c1*p+(tf-T/2)*c2*p

In [9]:
coutnonopti(40000,70000,0.1580,0.1230,86400,510,0.2,0.8)

38.78040000000001

Partie 3 : Etude avancée

N'ayant pas du tout compris l'alogorithme d'Uzawa et comment le mettre en place. J'ai décidé de faire mon propre algorithme pour le cas de deux voitures.

In [32]:
def newchoixtemps(t01,tf1,t02,tf2,pmax,SOCi1,SOCf1,SOCi2,SOCf2,T,Qmax):
    t11,t12=choixtemps(t01,tf1,pmax,SOCi1,SOCf1,T,Qmax)
    t21,t22=choixtemps(t02,tf2,pmax,SOCi2,SOCf2,T,Qmax)
    if t21>t12 or t11>t22 :
        return t11,t12,t21,t22
    else:
        if t11<t21:
            t21=(2/pmax)*(pmax*t12/2+pmax(t22-t12)-(SOCf2-SOCi2)*Qmax)
            t11=(1/pmax)*(pmax*t21+(pmax/2)(t12-t21)-(SOCf1-SOCi1)*Qmax)
            return t11,t12,t21,t22
        else:
            t11=(2/pmax)*(pmax*t22/2+pmax(t12-t22)-(SOCf1-SOCi1)*Qmax)
            t21=(1/pmax)*(pmax*t11+(pmax/2)(t22-t11)-(SOCf2-SOCi2)*Qmax)
            return t11,t12,t21,t22
        

In [33]:
def p(t):
    if t<86400/2:
        return 0.1580
    else : 
        return 0.1230

def newcout(t01,tf1,t02,tf2,pmax,SOCi1,SOCf1,SOCi2,SOCf2,T,Qmax,c1,c2):
    c=0
    t11,t12,t21,t22=newchoixtemps(t01,tf1,t02,tf2,pmax,SOCi1,SOCf1,SOCi2,SOCf2,T,Qmax)
    if t21>t12 or t11>t22 :
       return c+cout(t11,t12,SOCi1,SOCf1,Qmax,c1,c2,T,pmax) +cout(t21,t22,SOCi2,SOCf2,Qmax,c1,c2,T,pmax)
    else:
        if t11<t21:
            if p(t11)!=p(t21):
                c=c+pmax*p(t11)*(T/2-t11)+pmax*p(t21)*(t21-T/2)+pmax*p(t22)*(t22-t21)
                return c
            else :
                if p(t12)!=p(t21):
                    return c + pmax*p(t11)*(t21-t11)+(pmax/2)*p(t21)*(T/2-t21)+(pmax/2)*p(t12)(t12-T/2)+pmax*p(t22)*(t22-t12)
                else:
                    if p(t12)!=p(t22):
                        return c+pmax*p(t11)*(t21-t11)+(pmax/2)*p(t21)*(t12-t21)+pmax*p(t12)*(T/2-t12)+pmax*p(t22)*(t22-T/2)
                    else :
                        return p(t11)(pmax(t21-t11)+(pmax/2)(t12-t21)+pmax(t22-t12))
        else:
            if p(t11)!=p(t21):
                c=c+pmax*p(t21)*(T/2-t21)+pmax*p(t11)*(t11-T/2)+pmax*p(t12)*(t12-t11)
                return c
            else :
                if p(t22)!=p(t21):
                    return c + pmax*p(t21)*(t11-t21)+(pmax/2)*p(t11)*(T/2-t11)+(pmax/2)*p(t22)(t22-T/2)+pmax*p(t12)*(t12-t22)
                else:
                    if p(t22)!=p(t12):
                        return c+pmax*p(t21)*(t11-t21)+(pmax/2)*p(t11)*(t22-t11)+pmax*p(t22)*(T/2-t22)+pmax*p(t12)*(t12-T/2)
                    else :
                        return p(t21)(pmax(t11-t21)+(pmax/2)(t22-t11)+pmax(t12-t22))

                    
def imagecharge(t01,tf1,t02,tf2,pmax,SOCi1,SOCf1,SOCi2,SOCf2,T,Qmax):
    t11,t12,t21,t22=newchoixtemps(t01,tf1,t02,tf2,pmax,SOCi1,SOCf1,SOCi2,SOCf2,T,Qmax)
    Q1=[SOCi1*Qmax]
    Q2=[SOCi2*Qmax]
    if t21>t12 or t11>t22 :
        p1=(Qmax(SOCf1-SOCi1))/(t12-t11)
        for i in range (0,t12-t11+1):
            Q1.append(Q1[len(Q1)-1]+p1)
       
        p2=(Qmax(SOCf2-SOCi2))/(t22-t21)
        for i in range (0,t22-t21+1):
            Q2.append(Q2[len(Q2)-1]+p2)
        
    else: 
        if t11<t21:
            for i in range (0,t21-t11+1):
                Q1.append(Q1[len(Q1)-1]+pmax)
            for i in range (0,t12-t21+1):
                Q1.append(Q1[len(Q1)-1]+pmax/2)
            for i in range (0,t12-t21+1):
                Q2.append(Q2[len(Q2)-1]+pmax/2)
            for i in range (0,t22-t12+1):
                Q2.append(Q2[len(Q2)-1]+pmax)
    T1=[t11]
    for i in range (0,t12-t11+1):
        T1.append(T1[len(T1)-1]+1)
    T2=[t21]
    for i in range (0,t22-t21+1):
        T2.append(T2[len(T2)-1]+1)
    return Q1,T1,Q2,T2
    

In [34]:
newchoixtemps(20000,70000,30000,65000,6000,0.2,0.8,0.4,0.6,86400,510)

TypeError: 'int' object is not callable

-2